In [16]:
import math
import re
import sys
import string
from collections import Counter

import string
import random

import fuzzy
import nltk

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import codecs, difflib, Levenshtein, distance

debugYesGlobal = False

print (sys.version)

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [34]:
str1 = "sachin kumar went for a ride and was tired"
str2 = "sachin kumar came from a ride and felt tired "

str1 = str1.split()
str2 = str2.split()
matches = len(set(str1) & set(str2))
overlap_score = matches/(len(set(str1)) + len(set(str2)) - matches)

round(overlap_score,2)

0.5

### Let's Get Exact Overlap score

In [37]:
# NON-FUZZY OVERLAP SCORE. 
# Uses 'Set' to ignore duplicates. 
# Used for finding Exact match of 'PHOENETIC ENCODING'
# NOTE : It is case sensitive

def getExactOverlap (str1, str2) : 
    str1 = str1.split()
    str2 = str2.split()
    matches = len(set(str1) & set(str2))
    overlap_score = matches / (len(set(str1)) + len(set(str2)) - matches)
    return round(overlap_score, 2)
    
# Watch case        
print (getExactOverlap ('Hello world hello', 'Hello world'))
print (getExactOverlap ('Hello world Hello', 'Hello world'))

0.67
1.0


### Various edit distances from variety of algorithms

In [51]:
def getEditDistanceManyAlgos(sr3, sr4):
    diffl   = difflib.SequenceMatcher(None, sr3, sr4).ratio()
    lev     = Levenshtein.ratio(sr3, sr4) 
    sor     = 1 - distance.sorensen(sr3, sr4)
    jac     = 1 - distance.jaccard(sr3, sr4)
    exact   = getExactOverlap (sr3, sr4)
    print ("difflib:", round(diffl,2), "| Levenshtein:", round(lev,2), "| sorensen:", round(sor,2), "| jaccard", round(jac,2), "| exact", round(exact,2))

getEditDistanceManyAlgos ('Cukharlotte', 'edda Charlorte')
getEditDistanceManyAlgos ('Cukharlotte', 'Cukharlotte')

difflib: 0.64 | Levenshtein: 0.64 | sorensen: 0.8 | jaccard 0.67 | exact 0.0
difflib: 1.0 | Levenshtein: 1.0 | sorensen: 1.0 | jaccard 1.0 | exact 1.0


In [48]:
def getEditDistance (str1, str2, type) :
    score = 0
    if int(type) == 1 : score =  difflib.SequenceMatcher(None, str1, str2).ratio()
    if int(type) == 2 : score =  Levenshtein.ratio(str1, str2)
    if int(type) == 3 : score =  (1 - distance.sorensen(str1, str2))
    if int(type) == 4 : score =  (1 - distance.jaccard(str1, str2))
    if int(type) == 5 : score =  getExactOverlap (str1, str2)
    return round(score,2)

print (getEditDistance("Nike india","Nike USA", 5))
print (getEditDistance("india Nike","Nike India", 5))
print (getEditDistance("india Nikx","india Nike", 5))

0.33
0.33
0.33


In [54]:
getEditDistanceManyAlgos('nike', 'nikx')

difflib: 0.75 | Levenshtein: 0.75 | sorensen: 0.75 | jaccard 0.6 | exact 0.0


In [137]:
# https://github.com/Lilykos/pyphonetics
from pyphonetics import Soundex
soundex = Soundex()
print(soundex.phonetics('Rupert'))
#'R163'
print(soundex.phonetics('Robert'))
#'R163'
print(soundex.sounds_like('Robert', 'Rupert'))
#True

R163
R163
True


In [138]:
from pyphonetics import Metaphone
metaphone = Metaphone()
print(metaphone.phonetics('discrimination'))
#'TSKRMNXN'

TSKRMNXN


In [139]:
import fuzzy
dmeta = fuzzy.DMetaphone()
print ("Double Metaphone:", dmeta('fuzzy'))
#['FS', None]

print ("nysiis:", fuzzy.nysiis('fuzzy'))
#'FASY'


soundex = fuzzy.Soundex(4)
print ("Soundex:", soundex('fuzzy'))
#'F200'

Double Metaphone: [b'FS', None]
nysiis: FASY
Soundex: F200


In [140]:
# Simple Metaphone
def getPhoeneticDistanceMeta (str1, str2) :
    # Get the proper nouns as a string
    
    str1 = str1.split()
    str2 = str2.split()
 
    
    str1_meta = ' '.join([metaphone.phonetics(item) for item in str1])
    str2_meta = ' '.join([metaphone.phonetics(item) for item in str2])

    score =  getExactOverlap (str1_meta, str2_meta)
    
    if debugYesGlobal : 
        print ("\n")
        print ("1:", ' '.join(str1), "| 2:", ' '.join(str2))
        print ("1:", str1_meta, "| 2:", str2_meta)        
    
    return round(score,2)

# Testing
print(getPhoeneticDistanceMeta('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))

0.86


In [141]:
# Here 'Exact Match' and 'Soundex' are considered in sequence 
# (NOTE: 'Exact Match' just improves performance. 'Soundex' alone can handle exact match.)
def getPhoeneticDistanceSoundex (str1, str2) :
    # Get the proper nouns as a string
    
    # Lowercase
    str1 = str1.lower()
    str2 = str2.lower()    
    
    str1 = str1.split()
    str2 = str2.split()
    initial_count_both_strings = len(str1) + len(str2)
    if debugYesGlobal: print ("BEFORE REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    # Exact match count, in case-insensitive way
    matches_exact = len(set(str1) & set(str2))
    if debugYesGlobal: print ("Exact Matches (case?) : ", matches_exact)
    
    #removing exact matches
    for i in str1[:]:
      if i in str2:
          str1.remove(i)
          str2.remove(i)

    if debugYesGlobal: print ("AFTER REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    
    str1_Soundex_list = []
    for item in str1 :
        L = [soundex(item)]
        #L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str1Bloat += 1
        str1_Soundex_list = str1_Soundex_list + L
    #print ("str1Bloat=", str1Bloat) 
    
    str2_Soundex_list = []
    for item in str2 :
        L = [soundex(item)]
        #L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str2Bloat += 1
        str2_Soundex_list = str2_Soundex_list + L
    #print ("str2Bloat=", str2Bloat)
    
    matches_Soundex = 0
    items_covered1 = []
    # Find common matches based on Soundex
    for item in str1 :
        L = [soundex(item)]
        #L = list(filter(None.__ne__, L))
        
        # Exact match count, using Soundex codes
        if (len(set(str2_Soundex_list) & set(L))) > 0 : 
            matches_Soundex += 1 
            items_covered1.append(item)
    if debugYesGlobal: print ("Exact Soundex Matches (case?) : ", matches_Soundex)
    
    items_covered2 = []
    for item in str2 :
        L = [soundex(item)]
        #L = list(filter(None.__ne__, L))
        
        # Exact match count, using Soundex codes
        if (len(set(str1_Soundex_list) & set(L))) > 0 : 
            #matches_Soundex += 1 
            items_covered2.append(item)    
    
    if debugYesGlobal: 
        print ("Soundex items_covered1 : ", items_covered1)
        print ("Soundex items_covered2 : ", items_covered2)
    
    #Remove items covered from both strings
    if debugYesGlobal: print ("BEFORE REMOVING COMMON (Soundex) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    #removing ones matched with Soundex
    for i in str1[:]:
          if i in items_covered1:
                  str1.remove(i)
        
    for i in str2[:]:
          if i in items_covered2:
                  str2.remove(i)
    
    if debugYesGlobal: print ("AFTER REMOVING COMMON (Soundex) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
        
    total_matches = matches_exact + matches_Soundex
    score = total_matches / (initial_count_both_strings - total_matches)
       
    
    return round(score,2)


#print(getPhoeneticDistanceSoundex('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))
#print(getPhoeneticDistanceSoundex('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))

print(getPhoeneticDistanceSoundex('Schmidt Ripan Jackson McDonalds sauchages bun',' Smith Jackson Rupam McDonalds omlet'))

0.38


In [142]:
# Here 'Exact Match' and 'nysiis' are considered in sequence 
# (NOTE: 'Exact Match' just improves performance. 'nysiis' alone can handle exact match.)
def getPhoeneticDistanceNysiis (str1, str2) :
    # Get the proper nouns as a string
    
    # Lowercase
    str1 = str1.lower()
    str2 = str2.lower()    
    
    str1 = str1.split()
    str2 = str2.split()
    initial_count_both_strings = len(str1) + len(str2)
    if debugYesGlobal: print ("BEFORE REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    # Exact match count, in case-insensitive way
    matches_exact = len(set(str1) & set(str2))
    if debugYesGlobal: print ("Exact Matches (case?) : ", matches_exact)
    
    #removing exact matches
    for i in str1[:]:
      if i in str2:
          str1.remove(i)
          str2.remove(i)

    if debugYesGlobal: print ("AFTER REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    
    str1_nysiis_list = []
    for item in str1 :
        L = [fuzzy.nysiis(item)]
        #L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str1Bloat += 1
        str1_nysiis_list = str1_nysiis_list + L
    #print ("str1Bloat=", str1Bloat) 
    
    str2_nysiis_list = []
    for item in str2 :
        L = [fuzzy.nysiis(item)]
        #L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str2Bloat += 1
        str2_nysiis_list = str2_nysiis_list + L
    #print ("str2Bloat=", str2Bloat)
    
    matches_nysiis = 0
    items_covered1 = []
    # Find common matches based on nysiis
    for item in str1 :
        L = [fuzzy.nysiis(item)]
        #L = list(filter(None.__ne__, L))
        
        # Exact match count, using nysiis codes
        if (len(set(str2_nysiis_list) & set(L))) > 0 : 
            matches_nysiis += 1 
            items_covered1.append(item)
    if debugYesGlobal: print ("Exact nysiis Matches (case?) : ", matches_nysiis)
    
    items_covered2 = []
    for item in str2 :
        L = [fuzzy.nysiis(item)]
        #L = list(filter(None.__ne__, L))
        
        # Exact match count, using nysiis codes
        if (len(set(str1_nysiis_list) & set(L))) > 0 : 
            #matches_nysiis += 1 
            items_covered2.append(item)    
    
    if debugYesGlobal: 
        print ("nysiis items_covered1 : ", items_covered1)
        print ("nysiis items_covered2 : ", items_covered2)
    
    #Remove items covered from both strings
    if debugYesGlobal: print ("BEFORE REMOVING COMMON (nysiis) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    #removing ones matched with nysiis
    for i in str1[:]:
      if i in items_covered1:
          str1.remove(i)
        
    for i in str2[:]:
      if i in items_covered2:
          str2.remove(i)
    if debugYesGlobal: print ("AFTER REMOVING COMMON (nysiis) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
        
    total_matches = matches_exact + matches_nysiis
    score = total_matches / (initial_count_both_strings - total_matches)
       
    
    return round(score,2)


#print(getPhoeneticDistanceNysiis('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))
#print(getPhoeneticDistanceNysiis('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))

print(getPhoeneticDistanceNysiis('Schmidt Ripan Jackson McDonalds sauchages bun',' Smith Jackson Rupam McDonalds omlet'))

0.38


In [143]:
# Here 'Exact Match' and 'Double Metaphone' are considered in sequence 
# (NOTE: 'Exact Match' just improves performance. 'Double Metaphone' alone can handle exact match.)
def getPhoeneticDistanceDMeta2 (str1, str2) :
    # Get the proper nouns as a string
    
    # Lowercase
    str1 = str1.lower()
    str2 = str2.lower()    
    
    str1 = str1.split()
    str2 = str2.split()
    initial_count_both_strings = len(str1) + len(str2)
    if debugYesGlobal: print ("BEFORE REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    #str1_dmeta = ' '.join([' '.join(dmeta(item)) for item in str1])
    #str2_dmeta = ' '.join([' '.join(dmeta(item)) for item in str2])
    
    # Exact match count, in case-insensitive way
    matches_exact = len(set(str1) & set(str2))
    if debugYesGlobal: print ("Exact Matches (case?) : ", matches_exact)
    
    #removing exact matches
    for i in str1[:]:
      if i in str2:
          str1.remove(i)
          str2.remove(i)

    if debugYesGlobal: print ("AFTER REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    
    
    str1_dmeta_list = []
    for item in str1 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str1Bloat += 1
        str1_dmeta_list = str1_dmeta_list + L
    #print ("str1Bloat=", str1Bloat) 
    
    str2_dmeta_list = []
    for item in str2 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str2Bloat += 1
        str2_dmeta_list = str2_dmeta_list + L
    #print ("str2Bloat=", str2Bloat)
    
    matches_Dmeta = 0
    items_covered1 = []
    # Find common matches based on DMETAPHONE
    for item in str1 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        
        # Exact match count, using DMeta codes
        if (len(set(str2_dmeta_list) & set(L))) > 0 : 
            matches_Dmeta += 1 
            items_covered1.append(item)
    if debugYesGlobal: print ("Exact DMeta Matches (case?) : ", matches_Dmeta)
    
    items_covered2 = []
    for item in str2 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        
        # Exact match count, using DMeta codes
        if (len(set(str1_dmeta_list) & set(L))) > 0 : 
            #matches_Dmeta += 1 
            items_covered2.append(item)    
    
    if debugYesGlobal: 
        print ("DMETA items_covered1 : ", items_covered1)
        print ("DMETA items_covered2 : ", items_covered2)
    
    #Remove items covered from both strings
    if debugYesGlobal: print ("BEFORE REMOVING COMMON (DMETA) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    #removing ones matched with DMETA
    for i in str1[:]:
      if i in items_covered1:
          str1.remove(i)
        
    for i in str2[:]:
      if i in items_covered2:
          str2.remove(i)
    if debugYesGlobal: print ("AFTER REMOVING COMMON (DMETA) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
        
    total_matches = matches_exact + matches_Dmeta
    score = total_matches / (initial_count_both_strings - total_matches)
       
    
    return round(score,2)


print(getPhoeneticDistanceDMeta2('Schmidt  Jackson McDonalds sauchages bun',' Smith Jackson McDonalds omlet'))

0.5


In [144]:
import re

import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


#Clean text to extract words from raw data
# Raw product description text are run through :
#   a) Lowercasing all the letters 
#   b) remove punctuations, special characters, stopwords 
#   c) remove all numerical texts
def getStopwordRemovedString(rawData):  
    #res =  rawData.split(';')
    res =  re.split(r'\W+', rawData)
    # Lowercase
    res = ' '.join([word.lower() for word in res if (word.isalpha() and word not in stop_words)])
    #res = ' '.join([word for word in res if (word.isalpha() and word not in stop_words)])
    if debugYesGlobal: print (">>>", res)
    return res

In [145]:
# Here 'Exact Match', 'Double Metaphone' and 'Levenshtein' are considered in sequence
def getCombinedDistanceSequential (str1, str2) :
    # Get the proper nouns as a string
    
    # Lowercase
    #str1 = str1.lower()
    #str2 = str2.lower()
    
    str1 = getStopwordRemovedString(str1)
    str2 = getStopwordRemovedString(str2)
    if debugYesGlobal:     
        print (str1)
        print (str2)    
    
    str1 = str1.split()
    str2 = str2.split()
    initial_count_both_strings = len(str1) + len(str2)
    if debugYesGlobal: print ("BEFORE REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    #str1_dmeta = ' '.join([' '.join(dmeta(item)) for item in str1])
    #str2_dmeta = ' '.join([' '.join(dmeta(item)) for item in str2])
    
    # Exact match count, in case-insensitive way
    matches_exact = len(set(str1) & set(str2))
    if debugYesGlobal: print ("Exact Matches (case?) : ", matches_exact)
    
    #removing exact matches
    for i in str1[:]:
      if i in str2:
          str1.remove(i)
          str2.remove(i)

    if debugYesGlobal: print ("AFTER REMOVING COMMON >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    
    
    str1_dmeta_list = []
    for item in str1 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str1Bloat += 1
        str1_dmeta_list = str1_dmeta_list + L
    #print ("str1Bloat=", str1Bloat) 
    
    str2_dmeta_list = []
    for item in str2 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        #if len(L) == 2 : str2Bloat += 1
        str2_dmeta_list = str2_dmeta_list + L
    #print ("str2Bloat=", str2Bloat)
    
    matches_Dmeta = 0
    items_covered1 = []
    # Find common matches based on DMETAPHONE
    for item in str1 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        
        # Exact match count, using DMeta codes
        if (len(set(str2_dmeta_list) & set(L))) > 0 : 
            matches_Dmeta += 1 
            items_covered1.append(item)
    if debugYesGlobal: print ("Exact DMeta Matches (case?) : ", matches_Dmeta)
    
    items_covered2 = []
    for item in str2 :
        L = dmeta(item)
        L = list(filter(None.__ne__, L))
        
        # Exact match count, using DMeta codes
        if (len(set(str1_dmeta_list) & set(L))) > 0 : 
            #matches_Dmeta += 1 
            items_covered2.append(item)    
    
    
    if debugYesGlobal:
        print ("DMETA items_covered1 : ", items_covered1)
        print ("DMETA items_covered2 : ", items_covered2)
    
    #Remove items covered from both strings
    if debugYesGlobal: print ("BEFORE REMOVING COMMON (DMETA) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    #removing ones matched with DMETA
    for i in str1[:]:
      if i in items_covered1:
          str1.remove(i)
        
    for i in str2[:]:
      if i in items_covered2:
          str2.remove(i)
    if debugYesGlobal: print ("AFTER REMOVING COMMON (DMETA) >>>>1:", ' '.join(str1), "| 2:", ' '.join(str2))
    
    # Edit distance with the remaining:
    str1_for_editDist = ' '.join(str1)
    str2_for_editDist = ' '.join(str2)
    if debugYesGlobal: print ("\nstr_for_editDist1:", str1_for_editDist, "| str_for_editDist2:", str2_for_editDist)
    matches_Edit = getEditDistance (str1_for_editDist, str2_for_editDist, 2)
        
    total_matches = matches_exact + matches_Dmeta + matches_Edit
    
    if debugYesGlobal: print ("Exact Count:", matches_exact, "| Dmeta Count:", matches_Dmeta, "| Levenshtein Count/Score:", matches_Edit)
    score = total_matches / (initial_count_both_strings - total_matches)
    #print(getEditDistance (" ", " ", 2))
    
    return round(score,2)


#print(getPhoeneticDistanceDMeta('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))
#print(getPhoeneticDistanceDMeta('Michael likes to eat at McDonalds',' Jackson likes to eat at McDonalds Michael'))

print(getCombinedDistanceSequential('Schmidt  Jackson McDonalds sauchages bun',' Smith Jackson McDonalds omlet'))

0.53


In [146]:
string1 = 'Mikhail Jackson likes to eat is are at McDonalds'
#string2 = 'Michael Jackson likes to eat at McDonalds'
string2 = 'Michael likes to eat at McDonalds'
getCombinedDistanceSequential (string1, string2)  

0.8

In [147]:
show_metrics = True

# Combined Overlap Algorithm (levenshtein + metaphone)
def getCombinedDistanceExperimental (str1, str2) :
    
    str1 = getStopwordRemovedString(str1)
    str2 = getStopwordRemovedString(str2)
    #print (str1)
    #print (str2)
     
    metrics = []
    scores = []
    
    metrics.append("Soundex")
    scores.append (getPhoeneticDistanceSoundex(str1, str2))
    
    
    metrics.append("Nysiis")
    scores.append (getPhoeneticDistanceNysiis(str1, str2))

    metrics.append("Metaphone")
    scores.append (getPhoeneticDistanceMeta(str1, str2))
    metrics.append("DMetaphone")
    scores.append (getPhoeneticDistanceDMeta2(str1, str2))
    
    
    
    ### RIP :  BAD IDEA ### Get Levenshtein based score based on 'POS's other than nouns'
        
    # Get Levenshtein based overlap score with 'full string'
    #levenshteinScore = getEditDistance (str1, str2, 2)
    metrics.append("Leveshtein")
    scores.append (getEditDistance (str1, str2, 2))
    
    #metrics.append("Sequential(Exact>Dmeta>Levenshtein)")
    metrics.append("Combined(Exact>Dmeta>Levenshtein)")
    scores.append (getCombinedDistanceSequential(str1, str2))    
    
    
    if show_metrics : print (metrics)
    if debugYesGlobal: 
        print ("\n")
        print (metrics)
        print (scores)
    return scores #max(scores)
  

In [148]:
string1 = 'Mikhail Jackson likes to eat is are at McDonalds'
#string2 = 'Michael Jackson likes to eat at McDonalds'
string2 = 'Michael likes to eat at McDonalds'
getCombinedDistanceExperimental (string1, string2)  

['Soundex', 'Nysiis', 'Metaphone', 'DMetaphone', 'Leveshtein', 'Combined(Exact>Dmeta>Levenshtein)']


[0.8, 0.8, 0.5, 0.8, 0.81, 0.8]

In [149]:
string1 = 'Sachin Tendulkar drives a bike'
#string2 = 'Michael Jackson likes to eat at McDonalds'
string2 = 'Michael likes to eat at McDonalds'
getCombinedDistanceExperimental (string1, string2)    

['Soundex', 'Nysiis', 'Metaphone', 'DMetaphone', 'Leveshtein', 'Combined(Exact>Dmeta>Levenshtein)']


[0.0, 0.0, 0.0, 0.0, 0.36, 0.05]

In [150]:
getCombinedDistanceExperimental ('nike', 'nikx')

['Soundex', 'Nysiis', 'Metaphone', 'DMetaphone', 'Leveshtein', 'Combined(Exact>Dmeta>Levenshtein)']


[1.0, 0.0, 0.0, 0.0, 0.75, 0.6]

In [151]:
show_metrics = False

In [152]:
df = pd.read_csv ('Data_Sample_Overlap_scorer_v1_clean.csv',  encoding='latin1')

In [ ]:
display (df.head())

In [ ]:
# Take each row fro df and calculate all 6 Overlap scores
for index, row in df.iterrows():
    #print(row['system_description_3'], row['system_description_4'])
    #['Soundex', 'Nysiis', 'Metaphone', 'DMetaphone', 'Leveshtein', 'Combined(Exact>Dmeta>Levenshtein)']
    #[1.0, 0.0, 0.0, 0.0, 0.75, 0.6]
    scores_row = getCombinedDistanceExperimental (row['system_description_3'], row['system_description_4'])
        
    df.loc[index,'Metaphone']= scores_row[2]
    df.loc[index,'DMetaphone']= scores_row[3]
    df.loc[index,'Combined']= scores_row[5]
    
display (df.head()) 



In [ ]:
def TransformScoresToClasses (score):
    # Keep it between a and b
    a = -2
    b = 3
    
    xmin = 0
    xmax = 1
    return int(round(a + (b-a) * (score - xmin)/ (xmax-xmin)))
print(TransformScoresToClasses (0.87))

In [ ]:
def AnalyzeScores ():
    # Take each row fro df and calculate all 6 Overlap scores
    for index, row in df.iterrows():
        #print(row['system_description_3'], row['system_description_4'])
        #['Soundex', 'Nysiis', 'Metaphone', 'DMetaphone', 'Leveshtein', 'Combined(Exact>Dmeta>Levenshtein)']
        #[1.0, 0.0, 0.0, 0.0, 0.75, 0.6]
        scores_row = getCombinedDistanceExperimental (row['system_description_3'], row['system_description_4'])
        scores_row = list(map(TransformScoresToClasses, scores_row))

        df.loc[index,"Soundex"] = scores_row[0]
        df.loc[index,'Nysiis'] = scores_row[1]
        df.loc[index,'Metaphone']= scores_row[2]
        df.loc[index,'DMetaphone']= scores_row[3]
        df.loc[index,'Leveshtein']= scores_row[4] - 2 # [-1 if a,b = 1,4 | -2 if a,b = 1,5 | -1.5 if a,b = -2,-3]
        df.loc[index,'Combined']= scores_row[5]

    df['Average']= (df['Leveshtein'] + df['DMetaphone']) / 2



    #df.iloc[0:5, 5:8] # first 5 rows and 5th, 6th, 7th columns of dataframe 
    #display (df.head())
    
    # "Soundex","Nysiis","Metaphone","DMetaphone","Leveshtein", "Average", "Combined"
    n1 = 10
    n2 = 50
    df2 = df[n1:n2] #df.head(n)

    df2.reset_index().plot(x='index', y=["DMetaphone","Leveshtein"], figsize=(20,5), alpha=0.7, linestyle='-', linewidth=3, xticks=range(n1, n2)).legend(title='Scores', bbox_to_anchor=(1, 1))
    df2.reset_index().plot(x='index', y=["DMetaphone", "Combined"], figsize=(20,5), alpha=0.7, linestyle='-', linewidth=3, xticks=range(n1, n2)).legend(title='Scores', bbox_to_anchor=(1, 1))

AnalyzeScores ()

#### Figure 1 : Note that Levenshtein Based Overlap Score follows same pattern as Phonetic Overlap Score (Double Metaphone based). So, both cannot be kept (one should be removed). Levenshtein based overlap score is heavily truncated(dull). A good explanation being, potential Phonetic match is treated wrongly and prematurely as a spelling error. Sharp peaks and troughs indicate that Phonetic Score (Double Metaphone based) is less restrictive, comapared to Levenshtein. This is also cross-verified with real scenarios and Double Metaphone Documentation. So, if we have to keep any one, we will choose Phonetic Score (Double Metaphone). But still it misses true spelling errors. This is where Combined Overlap Score shines (Figure 2). 
#### Figure 2 : The Combined Overlap Score strikes a good balance by honouring the fine difference between Phonetic spelling variations and pure non-Phonetic spelling variations. 